## Installation

In [1]:
!pip install neo4j openai langchain tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.0/198.0 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━

## Connexion à la BD

In [ ]:
from langchain.graphs import Neo4jGraph

url = ""
username =""
password = ""
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

## FONCTIONS

In [ ]:
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
import openai
import os
import re
from openai import OpenAI

In [ ]:
def schema_extraction(graph):
  schema_visualisation = graph.query("call db.schema.visualization")
  rel_properties = graph.query("call db.schema.relTypeProperties")
  node_properties = graph.query("call db.schema.nodeTypeProperties")



  assistant = "Voici le schéma du graphe: " + str(schema_visualisation) + "Voici les propriété des différentes relations: " + str(rel_properties) + "Voici les propriétés des différents noeuds :" + str(node_properties)
  return schema_visualisation, rel_properties, node_properties


In [ ]:
def questionAnswering(question, graph, client):
  ##EXTRACTION SCHEMA DU GRAPH
  schema_visualisation, rel_properties, node_properties = schema_extraction(graph)
  ##GENERATION DE LA REQUETE CYPHER
  query_result = ""
  assistant = "Here is the graph schema: " + str(schema_visualisation) + "\n Here is the properties of the relations of the graph: " + str(rel_properties) + "\n Here is the properties of the nodes of the graph:" + str(node_properties)

  prompt_template = """
    give me a cypher Query to response to this:
    Question: {}
    """.format(question)
  messages = [ {"role": "system", "content": "You are an expert in generating Cypher Query"},
            {"role": "user", "content": prompt_template},
            {"role": "assistant", "content": assistant}
            ]

  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0
  )
  print(response.choices[0].message.content)

  ##EXTRACTION DE LA REQUETE CYPHER DE LA REPONSE DE GPT
  contenu_entre_backticks = re.findall(r"```(.*?)```", str(response.choices[0].message.content), re.DOTALL)
  if contenu_entre_backticks:
    for i, contenu in enumerate(contenu_entre_backticks, start=1):
        print(f"Contenu entre les triple backticks #{i}:")

        contenu = contenu.replace("<-", "-").replace("->", "-")
        print(contenu)
        query_result = query_result + str(graph.query(contenu))
  else:
    contenu = response.choices[0].message.content.replace("<-", "-").replace("->", "-")
    query_result =  str(graph.query(contenu))

  ##GENERATION DE LA REPONSE A LA QUESTION
  assistant = "This is the set of facts for the result of the question: \n" + query_result
  print(assistant)
  #question2 = "Using a Knowledge Graph, We extract this information:\n"+str(query_result)+".Using only these facts, responds to this question:"+question+ "the facts may contain multiple responses."
  question2 = "Generate a response to this question, using the given set of facts " + question
  print(question2)
  messages =  [{"role" : "system", "content" : "You are an expert in Answering generation using ONLY the given set of facts."},
              {"role": "user", "content": question2 },
              {"role": "assistant", "content": assistant}

              ]
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=0
  )
  return ("La réponse à la question est: " + response.choices[0].message.content)

## TEST

In [ ]:
question1 = "give me the set of persons who wrote and produced the same movie."
question2 = "What are the movies in which Zendaya has appeared?"
question3 = "tell me about spiderman, and the cast of the film far from home"
question4 = "Give me the movies that have the same production company"
question5 = "How many actors played in the movie Don't Look Up"

In [ ]:


os.environ['OPENAI_API_KEY'] = ""

client = OpenAI()


answer = questionAnswering(question3, graph, client)
print(answer)

To respond to the question "tell me about Spiderman, and the cast of the film Far From Home" using Cypher Query, you can use the following query:

```cypher
MATCH (m:Movie)-[:Cast]->(p:Person)
WHERE m.title CONTAINS 'Spiderman' OR m.title CONTAINS 'Far From Home'
RETURN m.title AS Movie, COLLECT(p.name) AS Cast
```

This query will match all movies related to Spiderman or Far From Home and retrieve the movie title along with the cast members.
Contenu entre les triple backticks #1:
cypher
MATCH (m:Movie)-[:Cast]-(p:Person)
WHERE m.title CONTAINS 'Spiderman' OR m.title CONTAINS 'Far From Home'
RETURN m.title AS Movie, COLLECT(p.name) AS Cast

This is the set of facts for the result of the question: 
[{'Movie': 'Spider-Man: Far From Home', 'Cast': ['Zendaya', 'Tom Holland', 'Samuel L. Jackson', 'Cobie Smulders', 'Jon Favreau', 'J. B. Smoove', 'Jacob Batalon', 'Martin Starr', 'Tony Revolori', 'Marisa Tomei', 'Jake Gyllenhaal']}]
Generate a response to this question, using the given set of 